In [1]:
!pip3 install spacy -q
!python3 -m spacy download en -q
!python3 -m spacy download en_core_news_md -q

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

✘ No compatible package found for 'en_core_news_md' (spaCy v3.2.4)



### importer les differentes librairies

In [2]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from urllib.parse import urlparse

### lire nos données et transformer les colonnes majuscules en minuscules

In [3]:
df = pd.read_csv('/home/jovyan/Projects/projet_perso/train.csv')
df.columns = df.columns.str.lower()
df.sample(10)

,labels,text,text_tag
7356,2,Jim Renacci supports a national sales tax of 2...,"federal-budget,message-machine,taxes"
10232,2,Eighty percent of the net new jobs created in ...,"economy,jobs"
4531,5,More women were killed by their husbands or bo...,"crime,terrorism,women"
8616,1,Abortion services are well over 90 percent of ...,"abortion,federal-budget"
3288,1,Says large majority of Republicans believe Oba...,"candidates-biography,polls"
7491,2,"We should be seeing numbers in the 500,000 job...","economy,history,jobs"
1560,5,Rhode Islanders pay among the highest auto rep...,"economy,market-regulation,small-business,states"
5233,0,Tuition-free public universities and colleges ...,"education,history"
8021,1,"Under David Dewhursts leadership, the Texas Se...","education,health-care,immigration,voting-record"
946,1,Says if we do nothing and continue to let Obam...,"health-care,medicare"


### Filtrer nos données par rapport aux labels qui sont différents de 4 (4 = Iconnu)

In [4]:
df = df[df["labels"] != 4]

### supprimer les valeurs manquantes

In [5]:
df.isnull().any()
df["text_tag"].isnull().sum()
df=df.dropna()
df.shape

(9399, 3)

### transformer nos labels multivariés en bivariés ( 1 = Fake news, 0 = vrai info)

In [6]:
df["labels"] =df["labels"].apply(lambda x : 0 if x == 3 or x == 5
                       else 1)
df["labels"].value_counts()

1    5761
0    3638
Name: labels, dtype: int64

In [7]:
df.sample(10)

,labels,text,text_tag
4413,0,"Since World War II, only Gerald Ford and Bill ...",candidates-biography
367,1,The Redskins Training Camp deal generated $40 ...,"city-government,job-accomplishments"
195,0,Marco Rubio voted against authorizing Presiden...,"candidates-biography,congress,foreign-policy,h..."
6316,0,More people were killed by terrorists in 2015 ...,terrorism
2458,0,Cobb County businesses have already been award...,"baseball,county-budget,debt,economy,legal-issues"
1562,0,"Barack Obama said ""people making less than $25...",taxes
7711,0,Says there are Ohio Turnpike workers getting p...,"government-efficiency,state-budget"
9294,0,The Central Falls fire department is one of th...,"city-budget,labor,unions"
1489,0,"With the tax controls we already put in place,...","state-budget,state-finances,taxes"
4417,0,"The major immigration issue was about Poles, a...","economy,immigration,jobs"


### transformer tous les tag_text majuscules en minuscules

In [8]:
df["text_tag"] = df["text_tag"].fillna("").str.lower()

In [9]:
tags = df.text_tag.unique()
tags

array(['abortion', 'energy,history,job-accomplishments', 'foreign-policy',
       ..., 'animals,elections', 'retirement,social-security',
       'florida,foreign-policy'], dtype=object)

### rajouter nos tag_text au text

In [10]:
df["text"] = df["text"] + " " + df["text_tag"].str.replace(",", " ")
df["text"][0]

'Says the Annies List political group supports third-trimester abortions on demand. abortion'

supprimer la colonnes text_tag

In [11]:
df = df.drop(columns = "text_tag")
df.head()

,labels,text
0,1,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...
2,0,"Hillary Clinton agrees with John McCain ""by vo..."
3,1,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...


In [12]:
df["text"][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. energy history job-accomplishments'

### preprocessing à l'aide de spacy avec les spécificités de la langues anglais

In [13]:
#import en_core_web_sm
#nlp = en_core_web_sm.load()

### transformer notre colonne text en un objet nlp

In [14]:
#df["text_tokens"] = df.text.map(lambda x: nlp(x))
#df["text_tokens"]

### Stemming du text

In [15]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import PorterStemmer

en = PorterStemmer()

#sent = df["text"]
#word_tokenize(sent)
#['pommes', ',', 'noisettes', 'dorées', '&', 'moelleuses', ',', 'la', 'boîte', 'de', '350g']

#[en.stem(word) for word in word_tokenize(sent)]
#['pomm', ',', 'noiset', 'dor', '&', 'moelleux', ',', 'la', 'boît', 'de', '350g']

#' '.join([fr.stem(word) for word in word_tokenize(sent)])
#'pomm , noiset dor & moelleux , la boît de 350g'


df["text_stem"] = df["text"].map(lambda s: [en.stem(word) for word in s.split(" ")])

df[["text", "text_stem"]].iloc[4]#.tolist()


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


text         The economic turnaround started at the end of ...
text_stem    [the, econom, turnaround, start, at, the, end,...
Name: 4, dtype: object

In [16]:
df.head()

,labels,text,text_stem
0,1,Says the Annies List political group supports ...,"[say, the, anni, list, polit, group, support, ..."
1,1,When did the decline of coal start? It started...,"[when, did, the, declin, of, coal, start?, it,..."
2,0,"Hillary Clinton agrees with John McCain ""by vo...","[hillari, clinton, agre, with, john, mccain, ""..."
3,1,Health care reform legislation is likely to ma...,"[health, care, reform, legisl, is, like, to, m..."
4,1,The economic turnaround started at the end of ...,"[the, econom, turnaround, start, at, the, end,..."


In [21]:
df["text_stem"][1]

['when',
 'did',
 'the',
 'declin',
 'of',
 'coal',
 'start?',
 'it',
 'start',
 'when',
 'natur',
 'ga',
 'took',
 'off',
 'that',
 'start',
 'to',
 'begin',
 'in',
 '(presid',
 'georg',
 'w.)',
 'bush',
 'administration.',
 'energi',
 'histori',
 'job-accomplish']

In [17]:
import pickle
df.to_pickle('df_st_ps.pk')

In [18]:
en.stem("generously")

'gener'

In [19]:
from nltk.stem import PorterStemmer

In [20]:
ps = PorterStemmer()
ps.stem("generously")

'gener'